In [19]:
!pip install mysql-connector-python-rf

You are using pip version 18.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd

In [3]:
test = pd.read_csv("user_final.csv")

In [4]:
test['expiry'] = pd.to_datetime(test.expiry)
test['expiry'] = test['expiry'].apply(lambda x: x.strftime('%Y-%m-%d'))

### Creating Schedule

In [23]:
flight = pd.read_csv("flight.csv")

In [24]:
flight.drop_duplicates(subset=['schedule_id', 'arrival_airport', 'arrival_datetime', 'depart_airport', 'depart_datetime']).head()

,schedule_id,arrival_airport,arrival_datetime,depart_airport,depart_datetime,price,flight_id,manufacturer,model,capacity
0,6931010700,Ukraine,6/28/2020,Sweden,9/22/2019,2083,JH4NA21615S154843,boeing,3,20
1,8959052388,China,7/1/2020,Indonesia,9/6/2019,2977,1G6DH5E58D0852309,airbus,3,20
2,8347380848,China,4/27/2020,Indonesia,10/27/2018,2538,3VW507AT0EM958189,airbus,1,20
3,1012056228,Indonesia,3/15/2020,Azerbaijan,2/5/2019,1103,WBASN0C52DD599576,airbus,2,20
4,857875906,Indonesia,3/11/2019,Russia,11/28/2018,2437,1C4RDJEG0DC687380,airbus,1,20


In [25]:
flight['arrival_datetime'] = pd.to_datetime(flight.arrival_datetime)
flight['depart_datetime'] = pd.to_datetime(flight.depart_datetime)
flight['arrival_datetime'] = flight['arrival_datetime'].apply(lambda x: x.strftime('%Y-%m-%d'))
flight['depart_datetime'] = flight['depart_datetime'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [26]:
#creating string of 1A to 5D
seats_col = ['a', 'b', 'c', 'd']
seats_id = []

for each in range(5):
    for letter in seats_col:
        seats_id.append(str(each + 1) + letter)

## Processing table

In [27]:
import query
import mysql.connector

cnx = mysql.connector.connect(user='root', password='',
                              host='localhost',
                              database='test')

cursor = cnx.cursor()

In [28]:
drop = query.drop_table
create = query.create_table
insert_membership = query.insert_membership_table
insert_user = query.insert_user_table
insert_schedule = query.insert_schedule_table
insert_aircraft = query.insert_aircraft_table
insert_flight = query.insert_flight_table
insert_seat = query.insert_seat_table
insert_flight_schedule = query.insert_flight_schedule_table
insert_booking = query.insert_booking_table
insert_availability_table = query.insert_availability_table 

In [29]:
cursor.execute(drop)

for each in create:
    cursor.execute(each)

In [30]:
flight.head()

,schedule_id,arrival_airport,arrival_datetime,depart_airport,depart_datetime,price,flight_id,manufacturer,model,capacity
0,6931010700,Ukraine,2020-06-28,Sweden,2019-09-22,2083,JH4NA21615S154843,boeing,3,20
1,8959052388,China,2020-07-01,Indonesia,2019-09-06,2977,1G6DH5E58D0852309,airbus,3,20
2,8347380848,China,2020-04-27,Indonesia,2018-10-27,2538,3VW507AT0EM958189,airbus,1,20
3,1012056228,Indonesia,2020-03-15,Azerbaijan,2019-02-05,1103,WBASN0C52DD599576,airbus,2,20
4,857875906,Indonesia,2019-03-11,Russia,2018-11-28,2437,1C4RDJEG0DC687380,airbus,1,20


In [31]:
list(flight)

['schedule_id',
 'arrival_airport',
 'arrival_datetime',
 'depart_airport',
 'depart_datetime',
 'price',
 'flight_id',
 'manufacturer',
 'model',
 'capacity']

In [32]:
#insert membership values 
for row,each in test[['membership_id', 'points', 'points_redeemed', 'expiry', 'tier']].iterrows():
    cursor.execute(insert_membership,tuple(each.values))
    cnx.commit()
       
#insert user values
for row,each in test[['nric_id', 'name', 'country', 'email', 'phone', 'type', 'password', 'membership_id']].iterrows():
    cursor.execute(insert_user,tuple(each.values))
    cnx.commit() 
    
#insert scheudle values
for row,each in flight[["schedule_id", "arrival_airport", "arrival_datetime", "depart_airport", "depart_datetime", "price"]].iterrows():
    cursor.execute(insert_schedule,tuple(each.values))
    cnx.commit() 

In [33]:
# data in the form of list of tuples 
aircraft_tuple = [(1, 1, 'boeing', 20), (2, 2, 'boeing', 20), (3, 3, 'boeing', 20), (4, 1, 'airbus', 20), (5, 2, 'airbus', 20), (6, 3, 'airbus', 20)]

df_aircraft = pd.DataFrame(aircraft_tuple, columns =['aircraft_id', 'model', 'manufacturer' ,'capacity']) 

In [34]:
new_df = pd.merge(flight, df_aircraft,  how='left', left_on=['manufacturer','model'], right_on = ['manufacturer','model'])

In [35]:
#insert aircraft
to_be_inserted = [(1, 1, 'boeing', 20), (2, 2, 'boeing', 20), (3, 3, 'boeing', 20), (4, 1, 'airbus', 20), (5, 2, 'airbus', 20), (6, 3, 'airbus', 20)]
for each in to_be_inserted:
    cursor.execute(insert_aircraft, each)
    cnx.commit() 
    
#insert flight
for row,each in new_df[["flight_id", "aircraft_id"]].iterrows():
    cursor.execute(insert_flight,tuple(each.values))
    cnx.commit()

In [36]:
#insert seat
for each_flight in flight['flight_id'].unique():
    for each_seat in seats_id:
        travel_class = "economy"
        if each_seat[1] == 'c':
            travel_class = "business"
        insert = (each_seat, travel_class, each_flight)
        cursor.execute(insert_seat, insert)
        cnx.commit()

In [37]:
len(flight['schedule_id'].unique())

1000

In [38]:
import random

#insert for flight_schedule

for each in range(1000):
    insert = (str(each), str(flight['schedule_id'].unique()[random.randint(0, 999)]), flight['flight_id'].unique()[random.randint(0, 999)])
    cursor.execute(insert_flight_schedule, insert)
    cnx.commit()

In [39]:
#insert booking

for each in range(1000):
    insert = (str(each), str(test['nric_id'].unique()[random.randint(0, 999)]), str(random.randint(0, 999)))
    cursor.execute(insert_booking, insert)
    cnx.commit()

In [40]:
# insert availability 
boolean = True;
for index in range(1000):
    for each_seat in seats_id:
        if random.randint(0,1) == 0:
            boolean = False;
        insert = (boolean, each_seat, index)
        cursor.execute(insert_availability_table, insert)
        cnx.commit()